# Design Patterns with First-Class Functions

In particular, in the context of languages with first-class functions, Norvig suggests rethinking the `Strategy, Command, Template Method, and Visitor patterns`. The general idea is: `you can replace instances of some participant class in these patterns with simple functions`, reducing a lot of boilerplate code. In this chapter, we will refactor Strategy using function objects, and discuss a similar approach to simplifying the Command pattern.

## Case Study: Refactoring Strategy

Strategy is a good example of a design pattern that can be simpler in Python if you leverage functions as first-class objects.

### Classic Strategy

The Strategy pattern is summarized like this in Design Patterns:

Define a family of algorithms, encapsulate each one, and make them interchangeable. Strategy lets the algorithm vary indepenently from clients that use it.

---

A clear example of Strategy pattern in the ecommerce domain is computing discounts to orders according to the attributes of the customer or inspection of the ordered items.

Consider an online store with these discount rules:
- Customers with 1,000 or more fidelity points get a global 5% discount per order
- A 10% discount is applied to each line item with 20 or more units in the same order
- Orders with at least 10  distinct items get a 7% global discount

For brevity assume that only one discount can be applied to an order

---

Context:
- Provides a service by `delegating some computation to interchangeable components that implement alternative algorithm`s. In ecommerce example, the context is an Order, which is configured to apply a promotional discount according to one of the several algorithsm

Strategy:
- The interface common to the components that implement the different algorithms. In our example, this role is played by an abstract class called Promotion

Concrete Strategy
- One of the concrete subclasses of Strategy. FidelityPromo, BulkPromo, and LargePromo are the three concrete strategies implemeneted.

---
In our example,
before instantiating an order, the system would somehow select a promotional discount
strategy and pass it to the Order constructor. The selection of the strategy is outside of
the scope of the pattern.

In [1]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
     
     def __init__(self, product, quantity, price) -> None:
         self.product = product
         self.quantity = quantity
         self.price = price
         
     def total(self):
          return self.price * self.quantity
     
     
class Order: # the context
     
     def __init__(self, customer: Customer, cart, promotion=None) -> None:
          self.customer = customer
          self.cart = list(cart)
          self.promotion = promotion
          
     def total(self):
          if not hasattr(self, '__total'):
               self.__total = sum(item.total() for item in self.cart)
          return self.__total
          
     def due(self):
          if self.promotion is None:
               discount = 0
          else:
               discount = self.promotion.discount(self)
          return self.total() - discount
     
     def __repr__(self) -> str:
         return f"{self.customer.name}, {self.total():.2f}, {self.due():.2f}"
    

class Promotion(ABC): # The Strategy: an abstract base class
     
     @abstractmethod
     def discount(self, order: Order):
          """Return discount as a positive dollar amount"""
          
          
class FidelityPromo(Promotion):
     """5% discount for each lineitem with 20 or more units"""
     
     def discount(self, order: Order):
         return order.total() * 0.05 if order.customer.fidelity == 1000 else 0 
    
    
class BulkItemPromo(Promotion):
     """10% discount for each lineitem with 20 or more units"""
     
     def discount(self, order: Order):
          discount = 0
          for item in order.cart:
              if item.quantity >= 20:
                   discount += item.total() * .1
          return discount         
          
          
class LargeOrderPromo(Promotion):
     """7% discount for orders with 10 or more distinct items"""
     
     def discount(self, order: Order):
          distinct_items = {item.product for item in order.cart}
          if len(distinct_items) >= 10:
              return order.total() * 0.7
          return 0


In [2]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)

cart = [LineItem('banana', 4, .5), 
  LineItem('apple', 10, 1.5),
  LineItem('watermellon', 5, 5.0)]

In [3]:
print(Order(joe, cart, FidelityPromo()))
print(Order(ann, cart, FidelityPromo()))

John Doe, 42.00, 42.00
Ann Smith, 42.00, 42.00


In [4]:
banana_cart = [LineItem('banana', 30, .5), 
LineItem('apple', 10, 1.5)]
Order(joe, banana_cart, BulkItemPromo()) 

John Doe, 30.00, 28.50

In [5]:
long_order = [LineItem(str(item_code), 1, 1.0) 
for item_code in range(10)]
Order(joe, long_order, LargeOrderPromo()) 


John Doe, 10.00, 3.00

In [6]:
Order(joe, cart, LargeOrderPromo())

John Doe, 42.00, 42.00

### Function-Oriented Strategy

Each concrete strategy aboce is a class with a single method, discount. Furthermore, the strategy instances have no state (no instance attributes). You could say they look a lot like plain functions.

Refactoring below:

In [7]:
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
        
    def total(self):
        return self.price * self.quantity
    
    
class Order: # the Context
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self) 
        return self.total() - discount
    
    def __repr__(self) -> str:
         return f"{self.customer.name}, {self.total():.2f}, {self.due():.2f}"
    

def fidelity_promo(order): 
 """5% discount for customers with 1000 or more fidelity points"""
 return order.total() * .05 if order.customer.fidelity >= 1000 else 0


def bulk_item_promo(order):
 """10% discount for each LineItem with 20 or more units"""
 discount = 0
 for item in order.cart:
     if item.quantity >= 20:
         discount += item.total() * .1
 return discount


def large_order_promo(order):
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

In [8]:
Order(joe, cart, fidelity_promo)

John Doe, 42.00, 42.00

“Strategy objectsoften make good flyweights.” A definition of the Flyweight in another part of that workstates: “A flyweight is a shared object that can be used in multiple contexts simultaneously.” The sharing is `recommended to reduce the cost of creating a new concrete strategy object when the same strategy is applied over and over again`


Often concrete strategies have no internal state; they only deal with data from the context. If that is the case, then by all means use plain old functions

### Choosing The Best Strategy: Simple Aproach

In [9]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo] 

def best_promo(order): 
    """Select best discount available"""
    return max(promo(order) for promo in promos)

In [10]:
Order(joe, long_order, best_promo)

John Doe, 10.00, 9.30

1. promos: a list of strategies implemented as functions
2. best_promo:  takes an instance of Order as arguement, as do the other *_promo functions
3. Using a generator expression, we apply each of the functions from promos to the order and return the maximum discount computed

### Finding Strategies in a Module

Modules in python are also first-class objects, and the standard library provides several functions to handle them. The built in `globals` is desricribed as the following in python.

gloabls()
- Return a dictionary representing the current global symbol table. This is always the dictionary of the current module ( inside a function or method, this is the module where it is defined, not the module from which it is called)

Below is a somewhat a hackish way of using  globals to help beest_promo automaticcally find the other  available *_promo functions

In [14]:
promos = [globals()[name] for name in globals()
          if name.endswith('_promo')
          and name != 'best_promo']

def best_promo(order):
    """Select best discount available
    """
    return max(promo(order) for promo in promos)

1. Iterate over each name in the dictionary returned by globals()
2. Select only names that end with the _promo suffix
3. Filter out the best_promo itself, to avoud an infinite recursion
4. No changes inside best_promo

Another way of collecting the available promotions would be to create a module and put all the strategy functions there, except for best_promo.


The function inspect.getmembers returns the attributes of an object—in this case, the promotions module—optionally filtered by a predicate (a boolean function). We use inspect.isfunction to get only the functions from the module.

Works regardless of the names given to the functions; all that matters is
that the promotions module contains only functions that calculate discounts given orders. Of course, this is an implicit assumption of the code. If someone were to create a function with a different signature in the promotions module, then best_promo wouldbreak while trying to apply it to an order

In [ ]:
# The promos list is built by introspection of a new promotions module
import inspect

promos = [func for name, func in
 inspect.getmembers(promotions, inspect.isfunction)]

def best_promo(order):
 """Select best discount available
 """
 return max(promo(order) for promo in promos)

## Command

The goal of Command is to decouple an object that invokes an operation (the Invoker) from the provider object that implements it (the Receiver). In the example from Design Patterns, each invoker is a menu item in a graphical application, and the receivers are the document being edited or the application itself.

The idea is to put a Command object between the two, implementing an interface with a single method, execute, which calls some method in the Receiver to perform the desired operation. That way the Invoker does not need to know the interface of the Receiver, and different receivers can be adapted through different Command subclasses. The Invoker is configured with a concrete command and calls its execute method to operate it.

Quoting from Gamma et al., “Commands are an object-oriented replacement for callbacks.” The question is: do we need an object-oriented replacement for callbacks? Sometimes yes, but not always.

Instead of giving the Invoker a Command instance, we can simply give it a function. Instead of calling command.execute(), the Invoker can just call command(). The Macro Command can be implemented with a class implementing `__call__`. Instances of Macro Command would be callables, each holding a list of functions for future invocation

In [18]:
class MacroCommand:
    
    def __init__(self, commands) -> None:
        self.commands = list(commands)
        
    def __call__(self) -> None:
        for command in self.commands:
            command()

1. Building a list from the commands arguements ensures that it is iterable and keeps a local copy of the comman references in each MacroCommand instance
2. When an instance of MacroCommand is invoked, each command in self.commands is called ins equence

More advanced uses of the Command pattern- to support undo, for example- may require more than a simple callback function. 

# Chapter Summary

In many cases, functions or callable objects provide a more natural way of implementing callbacks in Python than mimicking the Strategy or the Command patterns as described by Gamma, Helm, Johnson, and Vlissides. The refactoring of Strategy and the discussionof Command in this chapter are examples of a more general insight: sometimes you may encounter a design pattern or an API that requires that components implement an
interface with a single method, and that method has a generic-sounding name such as “execute”, “run”, or “doIt”. Such patterns or APIs often can be implemented with less boilerplate code in Python using first-class functions or other callables. The message from Peter Norvig’s design patterns slides is that the Command and Strategy patterns—along with Template Method and Visitor—can be made simpler or even “invisible” with first-class functions, at least for some applications of these patterns.